# Ewald sum

References:
- "Computer Simulation of Liquids" by Michael P. Allen and Dominic J. Tildesley (2nd edition, Oxford University Press, 2017)
- https://github.com/Allen-Tildesley

In [1]:
import numpy as np
import torch
from scipy.special import erfc

def distance(x):
    return torch.norm(x[None, :, :] - x[:, None, :], dim=-1)

def qpairs(q):
    return q[None, :]*q[:, None]

In [2]:
def rwald ( x, q, sigma=0.1178511 ):
    """Returns the r-space part of energy of ewald summation
    
    Arguments:
        x (float): position vectors (dim = n x 3)
        q (int): charges (dim = n)
        sigma (float): sqrt(variance), width of gaussian, ~ kappa
    
    Variables:
        n (int): number of particles
    
    Output:
        e_short (float): short range part of ewald potential e
    """
    epsilon = 8.854187817e-12
    #x = torch.tensor(x, dtype=np.float32)
    #q = torch.tensor(q, dtype=np.float32)
    n, d = list(x.size())
    assert n == q.shape[0], 'dimension error: q needs n entries'
    
    e_short = 0.
    
    r = distance(torch.Tensor(x))
    qiqj = qpairs(torch.Tensor(q))
    r_invers = r.clone()
    r_invers[r_invers!=0] = 1/r_invers[r_invers!=0]
    return torch.sum(
        qiqj * r_invers * erfc( r / (np.sqrt(2) * sigma) )
        ) / (2 * np.pi * epsilon)

In [3]:
charge = torch.tensor([-1., 2, -1, 3])
position = torch.tensor([[0., 0.], [0., 1.], [1., 1.], [1., 0.]])
#qiqj = qpairs(charge)
#print(qiqj)
#dist = distance(position)
#print(dist)
#dist[dist!=0] = 1/dist[dist!=0]
#print(qiqj*dist)
#print(torch.sum(qiqj))

In [4]:
print(rwald(position, charge))

tensor(-7.7363e-06)


In [5]:
a = np.array([[.1, .2, .4], [.8, 1.6, 3.2]])
print(a)
print(erfc(a*6))

[[0.1 0.2 0.4]
 [0.8 1.6 3.2]]
[[3.96143909e-001 8.96860218e-002 6.88513897e-004]
 [1.13521436e-011 5.52394460e-042 2.34001672e-162]]
